In [41]:
import torch



In [42]:
inputs=torch.tensor(
    [
     [0.43,0.15,0.89],#your
     [0.55,0.87,0.66],#journey
     [0.57,0.85,0.64],#starts
     [0.22,0.58,0.33],#with
     [0.77,0.25,0.10],#one
     [0.05,0.80,0.55],#step
    ]
)

In [43]:
w_q=torch.nn.Linear(inputs.shape[1],2,bias=False)
print(w_q)
w_k=torch.nn.Linear(inputs.shape[1],2,bias=False)
w_v=torch.nn.Linear(inputs.shape[1],2,bias=False)

Linear(in_features=3, out_features=2, bias=False)


In [47]:
queries=w_q(inputs)
keys=w_k(inputs)
values=w_v(inputs)

print(queries)

tensor([[-0.3535, -0.1955],
        [ 0.1574, -0.2035],
        [ 0.1563, -0.1833],
        [ 0.1585, -0.1544],
        [ 0.0919,  0.2323],
        [ 0.1709, -0.3557]], grad_fn=<MmBackward0>)


In [52]:
attn_score=queries@keys.T
print(attn_score)

tensor([[-0.1576, -0.2091, -0.2039, -0.1210, -0.0537, -0.1782],
        [ 0.0801,  0.1690,  0.1650,  0.1048,  0.0470,  0.1488],
        [ 0.0789,  0.1629,  0.1591,  0.1008,  0.0452,  0.1433],
        [ 0.0789,  0.1570,  0.1533,  0.0967,  0.0433,  0.1378],
        [ 0.0347,  0.0069,  0.0066, -0.0004, -0.0004,  0.0029],
        [ 0.0916,  0.2187,  0.2136,  0.1374,  0.0617,  0.1938]],
       grad_fn=<MmBackward0>)


In [58]:
attn_weight=torch.softmax(attn_score/keys.shape[1]**0.5,dim=1)
print(attn_weight)

tensor([[0.1661, 0.1602, 0.1608, 0.1705, 0.1788, 0.1637],
        [0.1621, 0.1726, 0.1721, 0.1649, 0.1583, 0.1701],
        [0.1624, 0.1723, 0.1719, 0.1649, 0.1586, 0.1700],
        [0.1628, 0.1721, 0.1716, 0.1649, 0.1588, 0.1698],
        [0.1698, 0.1665, 0.1665, 0.1656, 0.1656, 0.1660],
        [0.1595, 0.1745, 0.1738, 0.1647, 0.1561, 0.1714]],
       grad_fn=<SoftmaxBackward0>)


# Create Mask where the value above the diagonal is zero


In [89]:
#use pytorch tril function to diagonalize
context_length=inputs.shape[0]
mask1=torch.tril(torch.ones(context_length,context_length))
print(mask1)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [ ]:
#use element wise multiplication not matrix
masked_attn_score=attn_weight*mask1
print(masked_attn_score)

#but this attention score is not fully masked or optimal becaus the softmax normalization adds influence on its from the previous attn score so use upper negative infinity mask instead of zero

tensor([[0.1661, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1621, 0.1726, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1624, 0.1723, 0.1719, 0.0000, 0.0000, 0.0000],
        [0.1628, 0.1721, 0.1716, 0.1649, 0.0000, 0.0000],
        [0.1698, 0.1665, 0.1665, 0.1656, 0.1656, 0.0000],
        [0.1595, 0.1745, 0.1738, 0.1647, 0.1561, 0.1714]],
       grad_fn=<MulBackward0>)


In [77]:
#normalize the masked attention score  to sum up 1 in each row to get masked attn weight
row_sum=masked_attn_score.sum(dim=1,keepdim=True)
print(row_sum)
masked_attn_weight=masked_attn_score/row_sum
print(masked_attn_weight)

tensor([[0.1661],
        [0.3346],
        [0.5066],
        [0.6715],
        [0.8340],
        [1.0000]], grad_fn=<SumBackward1>)
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4843, 0.5157, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3206, 0.3402, 0.3393, 0.0000, 0.0000, 0.0000],
        [0.2425, 0.2563, 0.2556, 0.2456, 0.0000, 0.0000],
        [0.2036, 0.1996, 0.1996, 0.1986, 0.1986, 0.0000],
        [0.1595, 0.1745, 0.1738, 0.1647, 0.1561, 0.1714]],
       grad_fn=<DivBackward0>)


In [78]:
masked_context_vectore=masked_attn_weight@values
print(masked_context_vectore)

tensor([[-0.4688, -0.3531],
        [-0.5099, -0.1380],
        [-0.5192, -0.0738],
        [-0.4657, -0.0238],
        [-0.4096, -0.0585],
        [-0.4163, -0.0007]], grad_fn=<MmBackward0>)


# Negative infinity masking to avoid any influence of future attention

In [81]:
print(attn_score)

tensor([[-0.1576, -0.2091, -0.2039, -0.1210, -0.0537, -0.1782],
        [ 0.0801,  0.1690,  0.1650,  0.1048,  0.0470,  0.1488],
        [ 0.0789,  0.1629,  0.1591,  0.1008,  0.0452,  0.1433],
        [ 0.0789,  0.1570,  0.1533,  0.0967,  0.0433,  0.1378],
        [ 0.0347,  0.0069,  0.0066, -0.0004, -0.0004,  0.0029],
        [ 0.0916,  0.2187,  0.2136,  0.1374,  0.0617,  0.1938]],
       grad_fn=<MmBackward0>)


In [86]:
mask=torch.triu(torch.ones(context_length,context_length),diagonal=1)
print(mask)
masked=attn_score.masked_fill(mask.bool(),-torch.inf)
print(masked)

tensor([[0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])
tensor([[-0.1576,    -inf,    -inf,    -inf,    -inf,    -inf],
        [ 0.0801,  0.1690,    -inf,    -inf,    -inf,    -inf],
        [ 0.0789,  0.1629,  0.1591,    -inf,    -inf,    -inf],
        [ 0.0789,  0.1570,  0.1533,  0.0967,    -inf,    -inf],
        [ 0.0347,  0.0069,  0.0066, -0.0004, -0.0004,    -inf],
        [ 0.0916,  0.2187,  0.2136,  0.1374,  0.0617,  0.1938]],
       grad_fn=<MaskedFillBackward0>)


In [90]:
print(attn_score*mask1)

tensor([[-0.1576, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000],
        [ 0.0801,  0.1690,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0789,  0.1629,  0.1591,  0.0000,  0.0000,  0.0000],
        [ 0.0789,  0.1570,  0.1533,  0.0967,  0.0000,  0.0000],
        [ 0.0347,  0.0069,  0.0066, -0.0004, -0.0004,  0.0000],
        [ 0.0916,  0.2187,  0.2136,  0.1374,  0.0617,  0.1938]],
       grad_fn=<MulBackward0>)


In [87]:
masked_attn_weight=torch.softmax(masked/keys.shape[1],dim=1)
print(masked_attn_weight)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4889, 0.5111, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3243, 0.3382, 0.3375, 0.0000, 0.0000, 0.0000],
        [0.2447, 0.2544, 0.2540, 0.2469, 0.0000, 0.0000],
        [0.2025, 0.1997, 0.1997, 0.1990, 0.1990, 0.0000],
        [0.1616, 0.1722, 0.1717, 0.1653, 0.1592, 0.1700]],
       grad_fn=<SoftmaxBackward0>)


# Masking additional attention weight with Dropout

In [103]:
torch.manual_seed(123)
dropout=torch.nn.Dropout(0.5)
print(dropout)
ones=torch.ones(6,6)
print(ones)
print(dropout(ones))

Dropout(p=0.5, inplace=False)
tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]])
tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])


In [106]:
attn_dropout=dropout(attn_weight)
print(attn_dropout)

tensor([[0.0000, 0.3204, 0.0000, 0.0000, 0.3575, 0.3274],
        [0.0000, 0.3451, 0.0000, 0.3298, 0.0000, 0.3402],
        [0.3248, 0.3447, 0.3437, 0.0000, 0.3171, 0.3399],
        [0.3257, 0.0000, 0.0000, 0.0000, 0.3176, 0.0000],
        [0.3396, 0.3330, 0.3329, 0.0000, 0.3312, 0.0000],
        [0.0000, 0.0000, 0.3477, 0.0000, 0.0000, 0.3428]],
       grad_fn=<MulBackward0>)


In [ ]:
class Causal_Attention(torch.nn.Module):
    def __init__(self,in_d,out_d):
        super().__init__()
        self.w_q=torch.nn.Linear(in_d,out_d)
        self.w_k=torch.nn.Linear(in_d,out_d)
        self.w_v=torch.nn.Linear(in_d,out_d)
    
    def forward(self,input_embedding):
        queries=self.w_q(input_embedding)
        keys=self.w_k(input_embedding)
        values=self.w_v(input_embedding)

        attn_score=queries@keys.T
        attn_weight=torch.softmax(attn_score/keys[1]**0.5,dim=1)

        
        

        
        )


In [ ]:
x=[[1,2,3],[4,5,6],[7,8,9]]
y=torch.zeros(3,3)
for r in range(3):
    row=x[r]
    row[r+1:y.append(row)]=row[r+1:]*0
    y[r][r]
    
print(y)

   



tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
